<a href="https://colab.research.google.com/github/dineshuthayakumar/AIExperiments/blob/main/Chatbot_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KF Bot: Simple Chatbot on Kissflow Low Code/No Code Application Development

> Add blockquote



We will scrape the web content as well as our PDFs to answer user queries more effectively.


In [9]:
!pip install --upgrade --quiet  sentence-transformers gradio bs4 pypdf2 chromadb rapidocr-onnxruntime langchain langchain-community langchainhub langchain-openai langchain_huggingface

## Step 1: Scraping our web content and using that as the foundation.


In [10]:
# Load, chunk and index the contents of a blog/website.
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(web_path=("https://community.kissflow.com/t/q6h9qlf/creating-and-managing-subitems"))
docs = loader.load()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
docs[0]

Document(metadata={'source': 'https://community.kissflow.com/t/q6h9qlf/creating-and-managing-subitems', 'title': 'Creating and managing subitems - Using boards - Kissflow Community', 'description': 'Subitems help you break down complex tasks into multiple small tasks and track them efficiently.', 'language': 'en'}, page_content='\n\n\nCreating and managing subitems - Using boards - Kissflow Community\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\nCommunity\n\n\n\n\nDeveloper Documentation\nStatus\n\n\nHow to enable Javascript\n\n\n\n\n\n\n\n\n\n\nSign Up\n\n\nLogin\n\n\n\n\n\n\n\n\n\n\nWelcome to the Kissflow Community!\n\n\n\n\n\n\n\n\nSearch for answers here or ask a question\n\n\n\n\n\n\nProduct Documentation\n\n\nðŸŽ¬ Getting started\nðŸ“’ User guide\nðŸ”Ž Quick references\nðŸ’¼ Troubleshooting guide\n\n\n\nProduct Updates\n\n\nPre-release notes\nProduct announcements\nTips and tricks\n\n\nLatest Activity\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nPr

In [12]:
# Split and manage the texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [13]:
# Store embeddings in a Vector DB
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}
)
vectorstore_web = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever_web = vectorstore_web.as_retriever()

In [14]:
from google.colab import userdata
import os

# If you haven't already installed "openai" in your Colab environment, make sure to:
# !pip install openai

# From LangChain, import the OpenAI LLM
from langchain.chat_models import ChatOpenAI

# Replace this with your actual environment variable (if stored in colab form fields, etc.)
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAPIKey")

# Instantiate the OpenAI LLM (adjust model name if desired, e.g. "gpt-4" or "gpt-3.5-turbo")
llm = ChatOpenAI(model_name="gpt-4")

# Pull in the prompt from your prompt hub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# From your LangChain Core imports
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create your pipeline using OpenAI as the LLM
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_web.invoke("Can I add 200 sub items within a Kissflow Board item?")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'No, you cannot add 200 sub items within a Kissflow Board item. The maximum number of subitems you can add inside an item is 50.'